# DLPFC SEA-AD Oligodendrocyte Analysis

This notebook analyzes oligodendrocyte data from the SEA-AD dataset, specifically focusing on the dorsolateral prefrontal cortex (DLPFC) region and comparing different Braak stages.

## Objectives:
1. Load and inspect the OLIGO_DLPFC_SEA-AD.h5ad dataset
2. Check data normalization status
3. Filter for Braak stages: Reference, Braak 3, and Braak 6
4. Perform standard Scanpy processing if needed
5. Analyze cluster composition by Braak stage
6. Perform differential gene expression analysis
7. Visualize results with volcano plots


In [ ]:
# Import required libraries
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Configure matplotlib for modern Jupyter notebook display
# Use plt.rcParams instead of deprecated IPython.display.set_matplotlib_formats
plt.rcParams['figure.autolayout'] = True  # Prevents label overlapping
plt.rcParams['figure.dpi'] = 150  # Higher DPI for better resolution
plt.rcParams['figure.figsize'] = (10, 6)  # Default figure size
plt.rcParams['savefig.format'] = 'png'  # Default save format
plt.rcParams['savefig.dpi'] = 300  # High resolution for saved figures
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

# Set scanpy settings without using deprecated set_figure_params
sc.settings.verbosity = 3  # verbosity level
# Configure scanpy settings manually to avoid deprecated IPython function calls
sc.settings._vector_friendly = False
sc.settings.dpi = 150  # Match matplotlib DPI
sc.settings.dpi_save = 300
sc.settings.frameon = True
sc.settings.fontsize = 12
sc.settings.figsize = (10, 6)
sc.settings.color_map = 'viridis'

# Load environment variables
load_dotenv()

print("Libraries imported successfully!")
print("Matplotlib and Scanpy configured for modern Jupyter display")


In [ ]:
# Load data path from environment variables
data_path = os.getenv('DATA_PATH')
if data_path is None:
    raise ValueError("DATA_PATH not found in environment variables. Please check your .env file.")

# Construct the full path to the data file
data_file_path = os.path.join(data_path, 'OLIGO_DLPFC_SEA-AD.h5ad')

print(f"Data path: {data_path}")
print(f"Data file path: {data_file_path}")

# Check if file exists
if not os.path.exists(data_file_path):
    raise FileNotFoundError(f"Data file not found at: {data_file_path}")

print("Data file found!")


In [ ]:
# Load the AnnData object
print("Loading AnnData object...")
adata = sc.read_h5ad(data_file_path)

print(f"Dataset shape: {adata.shape}")
print(f"Number of cells: {adata.n_obs}")
print(f"Number of genes: {adata.n_vars}")

# Display basic information about the dataset
print("\nDataset info:")
print(f"Obs (cells) columns: {list(adata.obs.columns)}")
print(f"Var (genes) columns: {list(adata.var.columns)}")

# Check if there are any uns keys
if adata.uns:
    print(f"Unstructured metadata keys: {list(adata.uns.keys())}")


## Data Normalization Check

We need to check whether the data is normalized and if raw data is available for processing.


In [ ]:
# Check data normalization status
print("Checking data normalization status...")

# Check if raw data is available
has_raw = adata.raw is not None
print(f"Raw data available: {has_raw}")

# Check the main data matrix properties
print(f"\nMain data matrix:")
print(f"  - Data type: {type(adata.X)}")
print(f"  - Shape: {adata.X.shape}")
print(f"  - Data type of values: {adata.X.dtype if hasattr(adata.X, 'dtype') else 'Unknown'}")

# Check for normalized data in layers
print(f"\nAvailable layers: {list(adata.layers.keys())}")

# Check data distribution to infer normalization status
sample_data = adata.X[:1000, :1000].toarray() if hasattr(adata.X, 'toarray') else adata.X[:1000, :1000]
print(f"\nData statistics (sample of 1000x1000):")
print(f"  - Min: {np.min(sample_data):.2f}")
print(f"  - Max: {np.max(sample_data):.2f}")
print(f"  - Mean: {np.mean(sample_data):.2f}")
print(f"  - Median: {np.median(sample_data):.2f}")

# Determine if data appears to be normalized
is_normalized = np.max(sample_data) < 50 and np.mean(sample_data) < 10
print(f"\nData appears to be normalized: {is_normalized}")

# Store raw data if available and data is normalized
if has_raw and is_normalized:
    print("Raw count data is available and will be used for processing.")
    # Keep the current normalized data in X and raw in raw
elif not has_raw and not is_normalized:
    print("Data appears to be raw counts. Will proceed with normalization.")
    # This is raw count data, will need to normalize
else:
    print("Data status unclear. Will inspect further.")


## Filter Data for Braak Stages

Filter the dataset to include only cells from Braak stages: Reference, Braak 3, and Braak 6.


In [ ]:
# Check available Braak stage information
print("Checking Braak stage information...")

# Look for Braak-related columns
braak_columns = [col for col in adata.obs.columns if 'braak' in col.lower() or 'stage' in col.lower()]
print(f"Braak-related columns found: {braak_columns}")

# Display unique values in Braak-related columns
for col in braak_columns:
    unique_vals = adata.obs[col].unique()
    print(f"\n{col} unique values:")
    print(unique_vals)
    print(f"Count per value:")
    print(adata.obs[col].value_counts())

# Also check for any other potential grouping columns
print(f"\nAll available obs columns:")
for col in adata.obs.columns:
    print(f"  - {col}: {len(adata.obs[col].unique())} unique values")


In [ ]:
# Filter data for specific Braak stages
# Use the 'Braak stage' column specifically

# Define the exact Braak stages we want to keep
target_braak_stages = ["Reference", "Braak III", "Braak VI"]

# Use the specific 'Braak stage' column
braak_col = "Braak stage"
filtered_adata = adata.copy()

# Check if the 'Braak stage' column exists
if braak_col in adata.obs.columns:
    print(f"Using column '{braak_col}' for filtering")
    
    # Get unique values in the Braak stage column
    unique_vals = adata.obs[braak_col].unique()
    print(f"Available values in {braak_col}: {unique_vals}")
    
    # Find exact matches for our target stages
    matching_values = []
    for val in unique_vals:
        if str(val) in target_braak_stages:
            matching_values.append(val)
    
    if matching_values:
        print(f"Found matching values: {matching_values}")
        # Filter the data
        mask = adata.obs[braak_col].isin(matching_values)
        filtered_adata = adata[mask, :].copy()
        print(f"Filtered dataset shape: {filtered_adata.shape}")
        print(f"Original dataset shape: {adata.shape}")
        print(f"Cells removed: {adata.n_obs - filtered_adata.n_obs}")
        
        # Show distribution of remaining Braak stages
        print(f"\nDistribution after filtering:")
        print(filtered_adata.obs[braak_col].value_counts())
    else:
        print("No matching Braak stage values found. Keeping all data.")
        print("Available values:", unique_vals)
        print("Target values:", target_braak_stages)
else:
    print(f"Column '{braak_col}' not found in data.")
    print("Available Braak-related columns:", braak_columns)
    print("Available columns:", list(adata.obs.columns))

# Update adata to the filtered version
adata = filtered_adata


## Standard Scanpy Processing

If the data contains raw counts, perform standard Scanpy preprocessing steps.


In [ ]:
# Perform standard Scanpy processing if data appears to be raw counts
if not is_normalized or (has_raw and not is_normalized):
    print("Performing standard Scanpy preprocessing...")
    
    # Store raw data if not already stored
    if not has_raw:
        print("Storing raw count data...")
        adata.raw = adata
    
    # Basic filtering
    print("Calculating QC metrics...")
    sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)
    
    # Filter genes and cells
    print("Filtering genes and cells...")
    # Filter genes that are expressed in fewer than 3 cells
    sc.pp.filter_genes(adata, min_cells=3)
    
    # Filter cells that have fewer than 200 genes expressed
    sc.pp.filter_cells(adata, min_genes=200)
    
    # Remove cells with high mitochondrial gene expression
    adata.var['mt'] = adata.var_names.str.startswith('MT-')
    sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)
    
    # Remove cells with >20% mitochondrial genes
    adata = adata[adata.obs.pct_counts_mt < 20, :]
    
    print(f"After filtering: {adata.shape}")
    
    # Normalize and log-transform
    print("Normalizing and log-transforming...")
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    
    # Find highly variable genes
    print("Finding highly variable genes...")
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
    
    # Keep only highly variable genes for downstream analysis
    adata.raw = adata
    adata = adata[:, adata.var.highly_variable]
    
    print(f"After HVG filtering: {adata.shape}")
    
else:
    print("Data appears to be pre-processed. Skipping normalization steps.")

print("Preprocessing complete!")


## Clustering Analysis

Perform clustering and analyze cluster composition by Braak stage.


In [ ]:
# Perform clustering analysis
print("Performing clustering analysis...")

# Scale the data
print("Scaling data...")
sc.pp.scale(adata, max_value=10)

# Principal component analysis
print("Performing PCA...")
sc.tl.pca(adata, svd_solver='arpack')

# Compute neighborhood graph
print("Computing neighborhood graph...")
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

# Perform UMAP embedding
print("Computing UMAP embedding...")
sc.tl.umap(adata)

# Perform Leiden clustering
print("Performing Leiden clustering...")
sc.tl.leiden(adata, resolution=0.5)

# Check if clustering was successful
if 'leiden' in adata.obs.columns:
    n_clusters = len(adata.obs['leiden'].unique())
    print(f"Found {n_clusters} clusters")
    print("Cluster sizes:")
    print(adata.obs['leiden'].value_counts().sort_index())
else:
    print("Clustering failed - no 'leiden' column found")


In [ ]:
# Analyze cluster composition by Braak stage
print("Analyzing cluster composition by Braak stage...")

if braak_col and 'leiden' in adata.obs.columns:
    # Create a cross-tabulation
    cluster_braak_crosstab = pd.crosstab(adata.obs['leiden'], adata.obs[braak_col])
    print("\nCluster composition by Braak stage:")
    print(cluster_braak_crosstab)
    
    # Calculate proportions
    cluster_braak_prop = pd.crosstab(adata.obs['leiden'], adata.obs[braak_col], normalize='index')
    print("\nCluster composition by Braak stage (proportions):")
    print(cluster_braak_prop)
    
    # Visualize cluster composition
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Absolute counts
    cluster_braak_crosstab.plot(kind='bar', ax=axes[0], stacked=True)
    axes[0].set_title('Cluster Composition by Braak Stage (Counts)')
    axes[0].set_xlabel('Cluster')
    axes[0].set_ylabel('Number of Cells')
    axes[0].legend(title='Braak Stage', bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Proportions
    cluster_braak_prop.plot(kind='bar', ax=axes[1], stacked=True)
    axes[1].set_title('Cluster Composition by Braak Stage (Proportions)')
    axes[1].set_xlabel('Cluster')
    axes[1].set_ylabel('Proportion')
    axes[1].legend(title='Braak Stage', bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()
    
    # Statistical test for cluster composition differences
    from scipy.stats import chi2_contingency
    
    chi2, p_value, dof, expected = chi2_contingency(cluster_braak_crosstab)
    print(f"\nChi-square test for cluster composition differences:")
    print(f"Chi-square statistic: {chi2:.4f}")
    print(f"P-value: {p_value:.4e}")
    print(f"Degrees of freedom: {dof}")
    
    if p_value < 0.05:
        print("Significant difference in cluster composition between Braak stages (p < 0.05)")
    else:
        print("No significant difference in cluster composition between Braak stages (p >= 0.05)")

else:
    print("Cannot analyze cluster composition - missing Braak stage or cluster information")


## Differential Gene Expression Analysis

Perform differential gene expression analysis between Braak stages.


In [ ]:
# Perform differential gene expression analysis
print("Performing differential gene expression analysis...")

# Check if we have the necessary information
if braak_col and adata.raw is not None:
    print(f"Using Braak stage column: {braak_col}")
    print(f"Available Braak stages: {adata.obs[braak_col].unique()}")
    
    # Get unique Braak stages
    unique_braak_stages = adata.obs[braak_col].unique()
    
    # Perform pairwise comparisons
    comparisons = []
    
    # Find Reference and Braak 3/6 for comparison
    ref_stage = None
    braak3_stage = None
    braak6_stage = None
    
    for stage in unique_braak_stages:
        stage_str = str(stage).lower()
        if 'reference' in stage_str:
            ref_stage = stage
        elif 'braak' in stage_str and '3' in stage_str:
            braak3_stage = stage
        elif 'braak' in stage_str and '6' in stage_str:
            braak6_stage = stage
    
    print(f"Reference stage: {ref_stage}")
    print(f"Braak 3 stage: {braak3_stage}")
    print(f"Braak 6 stage: {braak6_stage}")
    
    # Perform comparisons
    if ref_stage and braak3_stage:
        print(f"\nComparing {ref_stage} vs {braak3_stage}")
        sc.tl.rank_genes_groups(adata, braak_col, groups=[braak3_stage], reference=ref_stage, 
                               method='wilcoxon', use_raw=True)
        comparisons.append(f"{ref_stage}_vs_{braak3_stage}")
    
    if braak3_stage and braak6_stage:
        print(f"\nComparing {braak3_stage} vs {braak6_stage}")
        sc.tl.rank_genes_groups(adata, braak_col, groups=[braak6_stage], reference=braak3_stage, 
                               method='wilcoxon', use_raw=True)
        comparisons.append(f"{braak3_stage}_vs_{braak6_stage}")
    
    print(f"Completed {len(comparisons)} comparisons")
    
else:
    print("Cannot perform DEG analysis - missing Braak stage information or raw data")
    print(f"Braak column: {braak_col}")
    print(f"Raw data available: {adata.raw is not None}")


## Volcano Plot Visualization

Create volcano plots to visualize the differential gene expression results.


In [ ]:
# Create volcano plots for differential gene expression results
def create_volcano_plot(adata, comparison_name, pval_thresh=0.05, logfc_thresh=0.5):
    """Create a volcano plot for differential gene expression results"""
    
    # Get the results
    result = adata.uns['rank_genes_groups']
    groups = result['names'].dtype.names
    
    if len(groups) == 0:
        print(f"No groups found for comparison {comparison_name}")
        return
    
    # Get the first group (should be the comparison group)
    group = groups[0]
    
    # Extract data
    names = result['names'][group]
    logfoldchanges = result['logfoldchanges'][group]
    pvals = result['pvals'][group]
    pvals_adj = result['pvals_adj'][group]
    
    # Create DataFrame
    df = pd.DataFrame({
        'gene': names,
        'logfc': logfoldchanges,
        'pval': pvals,
        'pval_adj': pvals_adj
    })
    
    # Remove NaN values
    df = df.dropna()
    
    # Calculate -log10(p-value)
    df['neg_log10_pval'] = -np.log10(df['pval'])
    
    # Define significance
    df['significant'] = (df['pval_adj'] < pval_thresh) & (np.abs(df['logfc']) > logfc_thresh)
    df['upregulated'] = df['significant'] & (df['logfc'] > 0)
    df['downregulated'] = df['significant'] & (df['logfc'] < 0)
    
    # Create volcano plot
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Plot non-significant genes
    ax.scatter(df[~df['significant']]['logfc'], 
               df[~df['significant']]['neg_log10_pval'], 
               c='gray', alpha=0.5, s=20, label='Not significant')
    
    # Plot upregulated genes
    if df['upregulated'].sum() > 0:
        ax.scatter(df[df['upregulated']]['logfc'], 
                   df[df['upregulated']]['neg_log10_pval'], 
                   c='red', alpha=0.7, s=30, label=f'Upregulated (n={df["upregulated"].sum()})')
    
    # Plot downregulated genes
    if df['downregulated'].sum() > 0:
        ax.scatter(df[df['downregulated']]['logfc'], 
                   df[df['downregulated']]['neg_log10_pval'], 
                   c='blue', alpha=0.7, s=30, label=f'Downregulated (n={df["downregulated"].sum()})')
    
    # Add threshold lines
    ax.axhline(y=-np.log10(pval_thresh), color='black', linestyle='--', alpha=0.5)
    ax.axvline(x=logfc_thresh, color='black', linestyle='--', alpha=0.5)
    ax.axvline(x=-logfc_thresh, color='black', linestyle='--', alpha=0.5)
    
    # Labels and title
    ax.set_xlabel('Log2 Fold Change')
    ax.set_ylabel('-Log10 P-value')
    ax.set_title(f'Volcano Plot: {comparison_name}')
    ax.legend()
    
    # Add gene labels for top genes
    top_genes = df.nlargest(5, 'neg_log10_pval')
    for _, row in top_genes.iterrows():
        ax.annotate(row['gene'], (row['logfc'], row['neg_log10_pval']), 
                   xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print(f"\nSummary for {comparison_name}:")
    print(f"Total genes tested: {len(df)}")
    print(f"Significantly upregulated: {df['upregulated'].sum()}")
    print(f"Significantly downregulated: {df['downregulated'].sum()}")
    print(f"Top 5 upregulated genes:")
    top_up = df[df['upregulated']].nlargest(5, 'logfc')
    for _, row in top_up.iterrows():
        print(f"  {row['gene']}: logFC={row['logfc']:.3f}, p-val={row['pval']:.2e}")
    
    print(f"Top 5 downregulated genes:")
    top_down = df[df['downregulated']].nsmallest(5, 'logfc')
    for _, row in top_down.iterrows():
        print(f"  {row['gene']}: logFC={row['logfc']:.3f}, p-val={row['pval']:.2e}")
    
    return df

# Create volcano plots for each comparison
if 'rank_genes_groups' in adata.uns:
    print("Creating volcano plots...")
    
    # Create plots for each comparison
    volcano_results = {}
    
    # Get the most recent comparison results
    result = adata.uns['rank_genes_groups']
    groups = result['names'].dtype.names
    
    for i, group in enumerate(groups):
        comparison_name = f"Comparison_{i+1}_{group}"
        print(f"\nCreating volcano plot for: {comparison_name}")
        
        # Create a temporary adata with just this comparison
        temp_adata = adata.copy()
        temp_result = {
            'names': result['names'][group:group+1],
            'logfoldchanges': result['logfoldchanges'][group:group+1],
            'pvals': result['pvals'][group:group+1],
            'pvals_adj': result['pvals_adj'][group:group+1]
        }
        temp_adata.uns['rank_genes_groups'] = temp_result
        
        volcano_df = create_volcano_plot(temp_adata, comparison_name)
        volcano_results[comparison_name] = volcano_df
        
else:
    print("No differential gene expression results found for volcano plots")


## Save Results

Create output folder and save all analysis results.


In [ ]:
# Create output directory
output_dir = Path("../output")
output_dir.mkdir(exist_ok=True)
print(f"Output directory: {output_dir.absolute()}")

# Save the processed AnnData object
processed_data_path = output_dir / "processed_DLPFC_SEA-AD_oligos.h5ad"
adata.write(processed_data_path)
print(f"Saved processed data to: {processed_data_path}")

# Save cluster composition results
if braak_col and 'leiden' in adata.obs.columns:
    cluster_braak_crosstab.to_csv(output_dir / "cluster_braak_composition_counts.csv")
    cluster_braak_prop.to_csv(output_dir / "cluster_braak_composition_proportions.csv")
    print("Saved cluster composition tables")

# Save differential gene expression results
if 'rank_genes_groups' in adata.uns:
    # Convert rank_genes_groups to DataFrame and save
    result = adata.uns['rank_genes_groups']
    
    # Save each comparison separately
    for group in result['names'].dtype.names:
        comparison_name = f"deg_results_{group}"
        
        # Create DataFrame for this comparison
        deg_df = pd.DataFrame({
            'gene': result['names'][group],
            'logfc': result['logfoldchanges'][group],
            'pval': result['pvals'][group],
            'pval_adj': result['pvals_adj'][group],
            'scores': result['scores'][group] if 'scores' in result else None
        })
        
        # Save to CSV
        deg_path = output_dir / f"{comparison_name}.csv"
        deg_df.to_csv(deg_path, index=False)
        print(f"Saved DEG results to: {deg_path}")

print("All results saved successfully!")


## Summary

This notebook has completed the following analyses:

1. ✅ Loaded and inspected the OLIGO_DLPFC_SEA-AD.h5ad dataset
2. ✅ Checked data normalization status and raw data availability
3. ✅ Filtered data for Braak stages: Reference, Braak 3, and Braak 6
4. ✅ Performed standard Scanpy processing (if raw counts detected)
5. ✅ Analyzed cluster composition by Braak stage with statistical testing
6. ✅ Performed differential gene expression analysis between Braak stages
7. ✅ Created volcano plots for visualization
8. ✅ Saved all results to the output folder

### Key Files Generated:
- `processed_DLPFC_SEA-AD_oligos.h5ad`: Processed AnnData object
- `cluster_braak_composition_counts.csv`: Cluster composition counts
- `cluster_braak_composition_proportions.csv`: Cluster composition proportions
- `deg_results_[comparison].csv`: Differential gene expression results for each comparison

### Next Steps:
- Review the volcano plots for biologically relevant genes
- Perform pathway enrichment analysis on significant DEGs
- Validate findings with additional datasets or experimental approaches
